* source of dataset: http://archive.ics.uci.edu/ml/datasets/online+retail

In [276]:
# libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import os
from multiprocessing import Pool
import re

pd.set_option("max_rows", 80)
pd.set_option("max_columns", None)

In [277]:
from numba import jit, autojit

In [278]:
df = pd.read_excel("Online_Retail.xlsx")

In [279]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [280]:
# create cancelled column with c_index
c_index = []
for i in range(len(df)):
    invoice = str(df.iloc[i,0])
    if 'C' in invoice or 'c' in invoice:
        c_index.append(i)

In [281]:
df.loc[c_index,'cancelled'] = 1

In [282]:
# create revenue column
df['revenue'] = df.Quantity * df.UnitPrice

In [283]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,cancelled,revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,NaN,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,NaN,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,NaN,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,NaN,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,NaN,20.34


In [284]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 10 columns):
InvoiceNo      541909 non-null object
StockCode      541909 non-null object
Description    540455 non-null object
Quantity       541909 non-null int64
InvoiceDate    541909 non-null datetime64[ns]
UnitPrice      541909 non-null float64
CustomerID     406829 non-null float64
Country        541909 non-null object
cancelled      9288 non-null float64
revenue        541909 non-null float64
dtypes: datetime64[ns](1), float64(4), int64(1), object(4)
memory usage: 41.3+ MB


In [285]:
# stockcodes with alphabet: create separate code for that
# str.isdigit() function

codes = [str(df.loc[i,'StockCode'])[len(df.loc[i,'StockCode'])-1] if str(df.loc[i,'StockCode']).isdigit() == False else np.nan for i in range(541909)]

In [286]:
# append codes to dataframe
df['code_alphabet'] = codes

In [287]:
df = df.drop('code_alphabet', axis = 1)

In [288]:
discounts = [1 if df.loc[i,'StockCode'] == 'D' else 0 for i in range(541909)]

In [289]:
# append discount information
df['discount'] = discounts

# view changes
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,cancelled,revenue,discount
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,NaN,15.30,0
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,NaN,20.34,0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,NaN,22.00,0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,NaN,20.34,0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,NaN,20.34,0


In [290]:
df.StockCode.value_counts()

85123A      2313
22423       2203
85099B      2159
47566       1727
20725       1639
84879       1502
22720       1477
22197       1476
21212       1385
20727       1350
22383       1348
22457       1280
23203       1267
POST        1256
22386       1251
22469       1239
22960       1229
21931       1214
22086       1210
22411       1202
20728       1197
22382       1192
22961       1182
22666       1180
23298       1179
22699       1138
22384       1137
23209       1135
82482       1129
22993       1111
22727       1107
22697       1085
23206       1084
22178       1072
20724       1068
23084       1067
20726       1061
22726       1026
21080       1015
23199       1009
            ... 
46776b         1
82613a         1
84612B         1
DCGS0004       1
90025E         1
90060B         1
20864          1
20863          1
90095          1
85160a         1
DCGS0068       1
90068          1
10134          1
90069          1
22275          1
35824B         1
90078          1
35603B        

In [291]:
# country continent mapping
continent_mapping = {'Australia': 'Oceania', 
                     'Cyprus': 'Middle East',
                     'Japan': 'Asia',
                     'USA': 'North America',
                     'Israel': 'Middle East',
                     'Hong Kong': 'Asia',
                     'Singapore': 'Asia',
                     'Canada': 'North America',
                     'United Arab Emirates': 'Middle East',
                     'RSA': 'Africa',
                     'Leabnon': 'Middle East',
                     'Brazil': 'South America',
                     'Bahrain': 'Middle East',
                     'Saudi Arabia': 'Middle East',
                     'Unspecified': 'Unspecified'
                    }

In [292]:
# assign continent
continents = [continent_mapping[df.loc[i,'Country']] if df.loc[i,'Country'] in continent_mapping else 'Europe' for i in range(541909)]

In [293]:
df['Continent'] = continents

# verify changes
df.Continent.value_counts()

Europe           537781
Oceania            1259
Middle East        1016
Asia                875
Unspecified         446
North America       442
Africa               58
South America        32
Name: Continent, dtype: int64

In [294]:
df.StockCode.value_counts()

85123A      2313
22423       2203
85099B      2159
47566       1727
20725       1639
84879       1502
22720       1477
22197       1476
21212       1385
20727       1350
22383       1348
22457       1280
23203       1267
POST        1256
22386       1251
22469       1239
22960       1229
21931       1214
22086       1210
22411       1202
20728       1197
22382       1192
22961       1182
22666       1180
23298       1179
22699       1138
22384       1137
23209       1135
82482       1129
22993       1111
22727       1107
22697       1085
23206       1084
22178       1072
20724       1068
23084       1067
20726       1061
22726       1026
21080       1015
23199       1009
            ... 
46776b         1
82613a         1
84612B         1
DCGS0004       1
90025E         1
90060B         1
20864          1
20863          1
90095          1
85160a         1
DCGS0068       1
90068          1
10134          1
90069          1
22275          1
35824B         1
90078          1
35603B        

In [295]:
df.loc[df.StockCode == 84879,]['Description'].value_counts()

ASSORTED COLOUR BIRD ORNAMENT    1501
damaged                             1
Name: Description, dtype: int64

In [296]:
sc = df.StockCode.value_counts().index.tolist()
dcgs = []
for i in sc:
    if 'DCGS' in str(i) or 'dcgs' in str(i):
        dcgs.append(i)

In [297]:
df[df.StockCode.isin(dcgs)]['Description'].value_counts()

GIRLS PARTY BAG                 13
BOYS PARTY BAG                  11
ebay                             5
BOXED GLASS ASHTRAY              4
SUNJAR LED NIGHT NIGHT LIGHT     2
HAYNES CAMPER SHOULDER BAG       1
CAMOUFLAGE DOG COLLAR            1
OOH LA LA DOGS COLLAR            1
Name: Description, dtype: int64

In [298]:
df[df.Description.isin(['damaged', 'Damaged', 'DAMAGED'])]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,cancelled,revenue,discount,Continent
142631,548620,84313B,Damaged,-32,2011-04-01 11:53:00,0.0,NaN,United Kingdom,NaN,-0.0,0,Europe
154122,549846,21100,Damaged,-6,2011-04-12 14:30:00,0.0,NaN,United Kingdom,NaN,-0.0,0,Europe
154656,549935,35915C,damaged,-43,2011-04-13 11:16:00,0.0,NaN,United Kingdom,NaN,-0.0,0,Europe
154799,549948,37488A,damaged,-1,2011-04-13 12:55:00,0.0,NaN,United Kingdom,NaN,-0.0,0,Europe
154835,549950,37487B,Damaged,-6,2011-04-13 12:57:00,0.0,NaN,United Kingdom,NaN,-0.0,0,Europe
154860,549952,37482P,Damaged,-93,2011-04-13 13:05:00,0.0,NaN,United Kingdom,NaN,-0.0,0,Europe
156618,550153,85167B,DAMAGED,-1,2011-04-14 14:44:00,0.0,NaN,United Kingdom,NaN,-0.0,0,Europe
166836,550954,47013A,damaged,-2,2011-04-21 16:18:00,0.0,NaN,United Kingdom,NaN,-0.0,0,Europe
170484,551302,48189,damaged,-18,2011-04-27 14:55:00,0.0,NaN,United Kingdom,NaN,-0.0,0,Europe
180618,552331,22667,damaged,-30,2011-05-09 10:24:00,0.0,NaN,United Kingdom,NaN,-0.0,0,Europe


In [299]:
# make description to all lowercase
df.Description = [str(i).lower() for i in df.Description]

In [300]:
# created 'damaged' column for all damaged items
df['damaged'] = [1 if i == 'damaged' else 0 for i in df.Description]

In [301]:
df[df.damaged == 1]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,cancelled,revenue,discount,Continent,damaged
142631,548620,84313B,damaged,-32,2011-04-01 11:53:00,0.0,NaN,United Kingdom,NaN,-0.0,0,Europe,1
154122,549846,21100,damaged,-6,2011-04-12 14:30:00,0.0,NaN,United Kingdom,NaN,-0.0,0,Europe,1
154656,549935,35915C,damaged,-43,2011-04-13 11:16:00,0.0,NaN,United Kingdom,NaN,-0.0,0,Europe,1
154799,549948,37488A,damaged,-1,2011-04-13 12:55:00,0.0,NaN,United Kingdom,NaN,-0.0,0,Europe,1
154835,549950,37487B,damaged,-6,2011-04-13 12:57:00,0.0,NaN,United Kingdom,NaN,-0.0,0,Europe,1
154860,549952,37482P,damaged,-93,2011-04-13 13:05:00,0.0,NaN,United Kingdom,NaN,-0.0,0,Europe,1
156618,550153,85167B,damaged,-1,2011-04-14 14:44:00,0.0,NaN,United Kingdom,NaN,-0.0,0,Europe,1
166836,550954,47013A,damaged,-2,2011-04-21 16:18:00,0.0,NaN,United Kingdom,NaN,-0.0,0,Europe,1
170484,551302,48189,damaged,-18,2011-04-27 14:55:00,0.0,NaN,United Kingdom,NaN,-0.0,0,Europe,1
180618,552331,22667,damaged,-30,2011-05-09 10:24:00,0.0,NaN,United Kingdom,NaN,-0.0,0,Europe,1


In [302]:
df[df.Quantity < 0]['StockCode'].value_counts()

M            244
22423        184
POST         126
22960         87
D             77
22720         76
S             61
21232         61
22699         54
22197         50
22666         47
82483         47
85099B        44
20725         44
85123A        43
21843         43
22697         42
21314         40
22776         40
23245         37
22698         36
23243         35
23169         35
84949         35
20914         34
22727         33
AMAZONFEE     32
22847         32
22467         32
22456         32
71477         31
84978         30
22784         29
21231         29
23110         28
84946         27
22138         27
85066         26
22726         25
37449         25
            ... 
22708          1
22713          1
22714          1
22731          1
90200C         1
22742          1
22743          1
22747          1
21385          1
22757          1
22761          1
22764          1
22765          1
21504          1
84227          1
22789          1
85188B         1
90177E        

In [303]:
df[df.StockCode == 22423]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,cancelled,revenue,discount,Continent,damaged
880,536477,22423,regency cakestand 3 tier,16,2010-12-01 12:27:00,10.95,16210.0,United Kingdom,NaN,175.20,0,Europe,0
936,536502,22423,regency cakestand 3 tier,2,2010-12-01 12:36:00,12.75,16552.0,United Kingdom,NaN,25.50,0,Europe,0
1092,536525,22423,regency cakestand 3 tier,2,2010-12-01 12:54:00,12.75,14078.0,United Kingdom,NaN,25.50,0,Europe,0
1155,536528,22423,regency cakestand 3 tier,1,2010-12-01 13:17:00,12.75,15525.0,United Kingdom,NaN,12.75,0,Europe,0
1197,536530,22423,regency cakestand 3 tier,1,2010-12-01 13:21:00,12.75,17905.0,United Kingdom,NaN,12.75,0,Europe,0
1341,536537,22423,regency cakestand 3 tier,2,2010-12-01 13:51:00,12.75,15922.0,United Kingdom,NaN,25.50,0,Europe,0
1390,536539,22423,regency cakestand 3 tier,2,2010-12-01 14:03:00,12.75,15165.0,United Kingdom,NaN,25.50,0,Europe,0
1576,536544,22423,regency cakestand 3 tier,40,2010-12-01 14:32:00,12.72,NaN,United Kingdom,NaN,508.80,0,Europe,0
2147,536562,22423,regency cakestand 3 tier,2,2010-12-01 15:08:00,12.75,13468.0,United Kingdom,NaN,25.50,0,Europe,0
2723,536592,22423,regency cakestand 3 tier,47,2010-12-01 17:06:00,12.72,NaN,United Kingdom,NaN,597.84,0,Europe,0


In [304]:
# cluster using recency,frequency, monetary
# so that we can profile other profiling variables

In [305]:
print(df.InvoiceDate.min())
print(df.InvoiceDate.max())

2010-12-01 08:26:00
2011-12-09 12:50:00


In [306]:
# average number of quantitiy
sum(df.Quantity) / len(df.Quantity)

9.5522495474332398

In [307]:
# max and min quantity
print(df.Quantity.min())
print(df.Quantity.max())

-80995
80995


In [308]:
df[(df.Quantity == -80995) | (df.Quantity == 80995)]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,cancelled,revenue,discount,Continent,damaged
540421,581483,23843,"paper craft , little birdie",80995,2011-12-09 09:15:00,2.08,16446.0,United Kingdom,NaN,168469.6,0,Europe,0
540422,C581484,23843,"paper craft , little birdie",-80995,2011-12-09 09:27:00,2.08,16446.0,United Kingdom,1.0,-168469.6,0,Europe,0


In [309]:
df.CustomerID.value_counts().sum()

406829

In [310]:
# divide between date and time for invoicedate
df['Date'] = [d.date() for d in df.InvoiceDate]
df['Time'] = [d.time() for d in df.InvoiceDate]

# verify split
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,cancelled,revenue,discount,Continent,damaged,Date,Time
0,536365,85123A,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,NaN,15.30,0,Europe,0,2010-12-01,08:26:00
1,536365,71053,white metal lantern,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,NaN,20.34,0,Europe,0,2010-12-01,08:26:00
2,536365,84406B,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,NaN,22.00,0,Europe,0,2010-12-01,08:26:00
3,536365,84029G,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,NaN,20.34,0,Europe,0,2010-12-01,08:26:00
4,536365,84029E,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,NaN,20.34,0,Europe,0,2010-12-01,08:26:00


In [311]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 15 columns):
InvoiceNo      541909 non-null object
StockCode      541909 non-null object
Description    541909 non-null object
Quantity       541909 non-null int64
InvoiceDate    541909 non-null datetime64[ns]
UnitPrice      541909 non-null float64
CustomerID     406829 non-null float64
Country        541909 non-null object
cancelled      9288 non-null float64
revenue        541909 non-null float64
discount       541909 non-null int64
Continent      541909 non-null object
damaged        541909 non-null int64
Date           541909 non-null object
Time           541909 non-null object
dtypes: datetime64[ns](1), float64(4), int64(3), object(7)
memory usage: 62.0+ MB


In [312]:
# missing values

In [314]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 15 columns):
InvoiceNo      541909 non-null object
StockCode      541909 non-null object
Description    541909 non-null object
Quantity       541909 non-null int64
InvoiceDate    541909 non-null datetime64[ns]
UnitPrice      541909 non-null float64
CustomerID     406829 non-null float64
Country        541909 non-null object
cancelled      9288 non-null float64
revenue        541909 non-null float64
discount       541909 non-null int64
Continent      541909 non-null object
damaged        541909 non-null int64
Date           541909 non-null object
Time           541909 non-null object
dtypes: datetime64[ns](1), float64(4), int64(3), object(7)
memory usage: 62.0+ MB


In [315]:
# missing customerid values
df[df.CustomerID.isnull()]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,cancelled,revenue,discount,Continent,damaged,Date,Time
622,536414,22139,nan,56,2010-12-01 11:52:00,0.00,NaN,United Kingdom,NaN,0.00,0,Europe,0,2010-12-01,11:52:00
1443,536544,21773,decorative rose bathroom bottle,1,2010-12-01 14:32:00,2.51,NaN,United Kingdom,NaN,2.51,0,Europe,0,2010-12-01,14:32:00
1444,536544,21774,decorative cats bathroom bottle,2,2010-12-01 14:32:00,2.51,NaN,United Kingdom,NaN,5.02,0,Europe,0,2010-12-01,14:32:00
1445,536544,21786,polkadot rain hat,4,2010-12-01 14:32:00,0.85,NaN,United Kingdom,NaN,3.40,0,Europe,0,2010-12-01,14:32:00
1446,536544,21787,rain poncho retrospot,2,2010-12-01 14:32:00,1.66,NaN,United Kingdom,NaN,3.32,0,Europe,0,2010-12-01,14:32:00
1447,536544,21790,vintage snap cards,9,2010-12-01 14:32:00,1.66,NaN,United Kingdom,NaN,14.94,0,Europe,0,2010-12-01,14:32:00
1448,536544,21791,vintage heads and tails card game,2,2010-12-01 14:32:00,2.51,NaN,United Kingdom,NaN,5.02,0,Europe,0,2010-12-01,14:32:00
1449,536544,21801,christmas tree decoration with bell,10,2010-12-01 14:32:00,0.43,NaN,United Kingdom,NaN,4.30,0,Europe,0,2010-12-01,14:32:00
1450,536544,21802,christmas tree heart decoration,9,2010-12-01 14:32:00,0.43,NaN,United Kingdom,NaN,3.87,0,Europe,0,2010-12-01,14:32:00
1451,536544,21803,christmas tree star decoration,11,2010-12-01 14:32:00,0.43,NaN,United Kingdom,NaN,4.73,0,Europe,0,2010-12-01,14:32:00


In [316]:
# since we need to groupby the customerid
# get rid of all rows without a customerid
# just to be safe, store it as another dataframe
# and view what's the common theme among the missing customerIDs

# create another dataframe for null ids
df_null_customer = df[df.CustomerID.isnull()]

# drop all rows without customer id
df.dropna(subset = ['CustomerID'], inplace = True)

In [334]:
df_null_customer.head()
df_null_customer.cancelled = df_null_customer.cancelled.fillna(0)

/Users/jk6653284/anaconda/lib/python3.6/site-packages/pandas/core/generic.py:2999: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [322]:
# verify rows have been dropped
df.CustomerID.isnull().sum()

0

In [332]:
# fill missing cancelled values as 0
df.cancelled = df.cancelled.fillna(0)

In [333]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 406829 entries, 0 to 541908
Data columns (total 15 columns):
InvoiceNo      406829 non-null object
StockCode      406829 non-null object
Description    406829 non-null object
Quantity       406829 non-null int64
InvoiceDate    406829 non-null datetime64[ns]
UnitPrice      406829 non-null float64
CustomerID     406829 non-null float64
Country        406829 non-null object
cancelled      406829 non-null float64
revenue        406829 non-null float64
discount       406829 non-null int64
Continent      406829 non-null object
damaged        406829 non-null int64
Date           406829 non-null object
Time           406829 non-null object
dtypes: datetime64[ns](1), float64(4), int64(3), object(7)
memory usage: 69.7+ MB


In [335]:
# download data as separate file to save trouble of loading this thing again and again

df.to_csv("online_retail_cleaned.csv")
df_null_customer.to_csv("online_retail_null_customers.csv")